In [ ]:
!pip install langchain_groq langchain_community chromadb
from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
import pandas as pd
import uuid
import chromadb

In [ ]:
llm = ChatGroq(
    temperature=0.5, 
    groq_api_key='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', 
    model_name="llama-3.3-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

That's an easy one!

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [3]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from  page of a website.
        Your job is to return information with appropriate keywords like deparments, faculty, academic
        buldings etc in json forma
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [7]:
res.content

'Here is the extracted information in JSON format:\n\n```\n{\n        "University": "Shahjalal University of Science and Technology",\n        "Departments": [\n            "School of Agriculture and Mineral Sciences",\n            "School of Applied Sciences and Technology",\n            "School of Life Sciences",\n            "School of Management & Business Administration",\n            "School of Physical Sciences",\n            "School of Social Sciences",\n            "Institute of Modern Languages",\n            "Institute of Information and Communication Technology"\n        ],\n        "Faculties": [\n            "Faculty of Agriculture and Mineral Sciences",\n            "Faculty of Applied Sciences and Technology",\n            "Faculty of Life Sciences",\n            "Faculty of Management & Business Administration",\n            "Faculty of Physical Sciences",\n            "Faculty of Social Sciences"\n        ],\n        "Academic Buildings": [\n            "Central Libra

In [4]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'University': 'Shahjalal University of Science and Technology',
 'Departments': ['School of Agriculture and Mineral Sciences',
  'School of Applied Sciences and Technology',
  'School of Life Sciences',
  'School of Management & Business Administration',
  'School of Physical Sciences',
  'School of Social Sciences',
  'Institute of Modern Languages',
  'Institute of Information and Communication Technology'],
 'Faculties': ['Faculty of Agriculture and Mineral Sciences',
  'Faculty of Applied Sciences and Technology',
  'Faculty of Life Sciences',
  'Faculty of Management & Business Administration',
  'Faculty of Physical Sciences',
  'Faculty of Social Sciences'],
 'Academic Buildings': ['Central Library',
  'Computer & Information Center',
  'Center of Excellence',
  'Institutional Quality Assurance Cell',
  'ICT Cell',
  'SUST Research Center'],
 'Research Centers': ['Center for Research, Testing and Consultancy',
  'Laboratory'],
 'Campus Life': ['Campus Life',
  'Cultural',
  'Re

In [5]:
import pandas as pd

In [6]:
df = pd.read_csv('350links.csv')
df

,Keywords,Links
0,Overview,https://docs.google.com/document/d/1p5rDNuK9mI...
1,Central Library,https://www.sust.edu/173/content-details/13/4


In [8]:
df.loc[len(df)] = ['schools and institutes', 'https://sust.edu/144/schools-departments/105']
df

,Keywords,Links
0,Overview,https://docs.google.com/document/d/1p5rDNuK9mI...
1,Central Library,https://www.sust.edu/173/content-details/13/4
2,schools and institutes,https://sust.edu/144/schools-departments/105
3,schools and institutes,https://sust.edu/144/schools-departments/105


In [9]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

for _, row in df.iterrows():
    collection.add(documents=row["Keywords"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

ValueError: The onnxruntime python package is not installed. Please install it with `pip install onnxruntime` in add.

In [11]:
import onnxruntime
print(onnxruntime.__version__)


ImportError: DLL load failed while importing onnxruntime_pybind11_state: A dynamic link library (DLL) initialization routine failed.

In [12]:
question = 'what are the department under applied science and technology school?'

In [13]:
links = collection.query(query_texts=question, n_results=1).get('metadatas', [])
links

ValueError: The onnxruntime python package is not installed. Please install it with `pip install onnxruntime` in query.

In [76]:
prompt_user = PromptTemplate.from_template(
        """
        ### Question:
        {question}
        
        ### INSTRUCTION:
        You are an assistant of an university. You will answer the questions given to you from the 
        provided links {links}. 
        Do not provide a preamble and the given links .Just only the required answer.
        ### QUESTION_ANSWER (NO PREAMBLE):
        
        """
        )

chain_answer = prompt_user | llm
res = chain_answer.invoke({"question": str(question), "links": links})
print(res.content)

The departments under the School of Applied Science and Technology are:

1. Department of Computer Science and Engineering
2. Department of Electrical and Electronic Engineering
3. Department of Environmental Science and Engineering
4. Department of Food Engineering and Technology
6. Department of Textile Engineering
7. Department of Industrial and Production Engineering
8. Department of Materials Science and Engineering
9. Department of Pharmacy


In [ ]:
!pip install fastapi nest-asyncio pyngrok uvicorn

In [13]:
!ngrok config add-authtoken 2xtQ4VTsnytY20U2JGk9ck2dqzO_3dTrMvdoZUu5jUJVJucJn

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [29]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()

# middlewares
app.add_middleware(
    CORSMiddleware, # https://fastapi.tiangolo.com/tutorial/cors/
    allow_origins=['*'], # wildcard to allow all, more here - https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/Access-Control-Allow-Origin
    allow_credentials=True, # https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/Access-Control-Allow-Credentials
    allow_methods=['*'], # https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/Access-Control-Allow-Methods
    allow_headers=['*'], # https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/Access-Control-Allow-Headers
)

items = ['apple']

@app.get("/")
async def root():
    return {"Hello World"}
@app.post("/items")
def create_item(item:str):
    items.append(item)
    return items
@app.get("/items")
def get_items():
    return items

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn


# specify a port
port = 8000
ngrok_tunnel = ngrok.connect(port)

# where we can visit our fastAPI app
print('Public URL:', ngrok_tunnel.public_url)


nest_asyncio.apply()

# finally run the app
uvicorn.run(app, port=port)

Public URL: https://89ca-34-168-149-80.ngrok-free.app


INFO:     Started server process [31]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     103.66.231.8:0 - "GET / HTTP/1.1" 200 OK
INFO:     103.66.231.8:0 - "GET /items HTTP/1.1" 200 OK


In [20]:
@app.get('/')
async def root():
    return {"Hell"}